In [3]:
from My_function_foody import *

In [4]:
urllib3.disable_warnings(InsecureRequestWarning)

In [5]:
Link_page = [
    "https://www.foody.vn/ha-noi/sang-trong?CategoryGroup=food&c=sang-trong", 
    "https://www.foody.vn/ha-noi/buffet?CategoryGroup=food&c=buffet",
    "https://www.foody.vn/ha-noi/nha-hang?CategoryGroup=food&c=nha-hang",
    "https://www.foody.vn/ha-noi/an-vat-via-he?CategoryGroup=food&c=an-vat-via-he",
    "https://www.foody.vn/ha-noi/an-chay?CategoryGroup=food&c=an-chay", 
    "https://www.foody.vn/ha-noi/cafe?CategoryGroup=food&c=cafe",
    "https://www.foody.vn/ha-noi/quan-an?CategoryGroup=food&c=quan-an", 
    "https://www.foody.vn/ha-noi/bar-pub?CategoryGroup=food&c=bar-pub", 
    "https://www.foody.vn/ha-noi/quan-nhau?CategoryGroup=food&c=quan-nhau",
    "https://www.foody.vn/ha-noi/tiec-tan-noi?CategoryGroup=food&c=tiec-tan-noi", 
    "https://www.foody.vn/ha-noi/beer-club?CategoryGroup=food&c=beer-club", 
    "https://www.foody.vn/ha-noi/tiem-banh?CategoryGroup=food&c=tiem-banh", 
    "https://www.foody.vn/ha-noi/foodcourt?CategoryGroup=food&c=Foodcourt",
    "https://www.foody.vn/ha-noi/shop-online?CategoryGroup=food&c=shop-online", 
    "https://www.foody.vn/ha-noi/giao-com-van-phong?CategoryGroup=food&c=giao-com-van-phong"
             ]

# Thu thập Link quán từ trang chủ foody


In [ ]:
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--start-maximized")  
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get("https://www.foody.vn/ha-noi")
time.sleep(300)
links = []

for link in Link_page:
    driver.get(link)
    time.sleep(2)  
    while True:
        scroll_until_end(driver)
        time.sleep(2)
        
        if not click_load_more(driver):
            break 
    
    click_close_button(driver)
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    row_items = soup.find_all('div', class_='row-item filter-result-item')
    for item in row_items:
        a_tag = item.find('a', attrs={'href': True})
        if a_tag:
            print(a_tag['href'])
            links.append(a_tag['href'])

    

In [7]:
write_file_txt(links, "extracted_links.txt")

In [ ]:
links = read_file_txt("extracted_links.txt")

# thu thập từ các link quán từ các thương hiệu có nhiều quán ăn 


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--start-maximized")  # Mở trình duyệt ở chế độ toàn màn hình

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

base_url = "https://www.foody.vn"  # URL cơ sở

result_dict = {}

for i in links:
    if "/thuong-hieu/" in i: 
        full_url = base_url + i  
        driver.get(full_url)  
        time.sleep(2)

        href_list = []  

        while True:
            scroll_height = 100  
            current_scroll_position = 0
            end_of_page = False

            while not end_of_page:
                driver.execute_script(f"window.scrollBy(0, {scroll_height});") 
                time.sleep(0.5)
                current_scroll_position += scroll_height

                new_height = driver.execute_script("return document.body.scrollHeight")
                if current_scroll_position >= new_height:
                    end_of_page = True

                try:
                    load_more_button = driver.find_element(By.CSS_SELECTOR, ".pn-loadmore .fd-btn-more")
                    load_more_button.click()
                    time.sleep(2)  
                    break  
                except Exception:
                    pass

            ldc_items = driver.find_elements(By.CLASS_NAME, "ldc-item.ng-scope")
            
            for item in ldc_items:
                try:
                    a_tag = item.find_element(By.TAG_NAME, "a")
                    href = a_tag.get_attribute("href") 
                    if href.startswith(base_url): 
                        href = href.replace(base_url, "")
                    if href not in href_list: 
                        href_list.append(href)
                        print(href)
                except Exception as e:
                    print(f"Error while extracting href: {e}")

            try:
                driver.find_element(By.CSS_SELECTOR, ".pn-loadmore .fd-btn-more")
            except Exception:
                break

        result_dict[i] = href_list


# Thu thập dữ liệu từ quán ăn

In [ ]:
# Cấu hình Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--start-maximized")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

base_url = "https://www.foody.vn"  # URL cơ sở

# Cấu trúc dữ liệu nhà hàng
List_restaurant_data = []
MAX_RETRIES = 3  # Số lần thử lại tối đa

# Thu thập dữ liệu từ các liên kết
for link in links:
    full_url = base_url + link
    retries = 0
    data_extracted = False

    while retries < MAX_RETRIES and not data_extracted:
        try:
            driver.get(full_url)
            time.sleep(random.randint(1, 2))  # Đợi trang tải hoàn toàn

            # Lấy source HTML của trang
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")

            # Khởi tạo cấu trúc dữ liệu nhà hàng
            restaurant = {
                "Image": "",
                "Link": full_url,
                "name": None,
                "categories": [],
                "cuisine": None,
                "audiences": [],
                "ratings": {},
                "comments": {},
                "address": {
                    "street": None,
                    "locality": None,
                    "region": None
                },
                "opening_hours": None,
                "price_range": None,
                "menu": []
            }

            # Lấy URL ảnh chính
            image_tag = soup.select_one(".main-image .img img")
            if image_tag and image_tag.get("src"):
                restaurant["Image"] = image_tag["src"].strip()

            # Tên nhà hàng
            name = soup.select_one("h1[itemprop='name']")
            restaurant["name"] = name.text.strip() if name else None

            # Các loại hình phụ (categories)
            restaurant["categories"] = [a.text.strip() for a in soup.select(".category-items a")]

            # Phong cách món ăn (cuisine)
            cuisine = soup.select_one(".cuisines-list a")
            restaurant["cuisine"] = cuisine.text.strip() if cuisine else None

            # Đối tượng phù hợp (audiences)
            audiences = soup.select_one(".audiences")
            if audiences:
                audiences_text = audiences.text.strip()
                audiences_list = audiences_text.replace("-", "").replace("\xa0", "").split(",")
                restaurant["audiences"] = [aud.strip() for aud in audiences_list]

            # Địa chỉ
            address_container = soup.select_one(".res-common-add")
            if address_container:
                street_address = address_container.select_one("span[itemprop='streetAddress']")
                restaurant["address"]["street"] = street_address.text.strip() if street_address else None

                address_locality = address_container.select_one("span[itemprop='addressLocality']")
                restaurant["address"]["locality"] = address_locality.text.strip() if address_locality else None

                address_region = address_container.select_one("span[itemprop='addressRegion']")
                restaurant["address"]["region"] = address_region.text.strip() if address_region else None

            # Giờ mở cửa
            opening_times = soup.select(".micro-timesopen span")
            if len(opening_times) >= 2:
                restaurant["opening_hours"] = opening_times[1].find_next_sibling("span").text.strip()

            # Giá
            price_range = soup.select_one(".res-common-minmaxprice span[itemprop='priceRange']")
            restaurant["price_range"] = price_range.text.strip() if price_range else None

            # Lấy đánh giá và bình luận
            ratings, comments = extract_ratings_and_comments(soup)
            restaurant["ratings"] = ratings
            restaurant["comments"] = comments
            # Menu
            menu_container = soup.select(".delivery-dishes-item")
            for item in menu_container:
                name_tag = item.select_one(".title-name-food .title-name")
                price_tag = item.select_one(".price")
                image_tag = item.select_one(".delivery-dishes-item-left img")

                if name_tag and price_tag and image_tag:
                    menu_item = {
                        "name": name_tag.text.strip(),
                        "price": price_tag.text.strip(),
                        "image_url": image_tag["src"].strip()
                    }
                    restaurant["menu"].append(menu_item)

            # Thêm vào danh sách
            List_restaurant_data.append(restaurant)
            data_extracted = True

        except Exception as e:
            print(f"Retry {retries + 1}/{MAX_RETRIES} - Lỗi: {e}")
            retries += 1
            time.sleep(random.randint(1, 2))  # Đợi trước khi thử lại

    if not data_extracted:
        print(f"Không thể thu thập dữ liệu từ {full_url} sau {MAX_RETRIES} lần thử.")

# Đóng trình duyệt
driver.quit()

NameError: name 'Options' is not defined

In [ ]:
# Cấu hình Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--start-maximized")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


# Cấu trúc dữ liệu nhà hàng
List_restaurant_data = []
MAX_RETRIES = 3  # Số lần thử lại tối đa

# Thu thập dữ liệu từ các liên kết
for link in links:
    retries = 0
    data_extracted = False

    while retries < MAX_RETRIES and not data_extracted:
        try:
            driver.get(link)
            time.sleep(random.randint(1, 2))  # Đợi trang tải hoàn toàn

            # Lấy source HTML của trang
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")

            # Khởi tạo cấu trúc dữ liệu nhà hàng
            restaurant = {
                "Image": "",
                "Link": link,
                "name": None,
                "categories": [],
                "cuisine": None,
                "audiences": [],
                "ratings": {},
                "comments": {},
                "address": {
                    "street": None,
                    "locality": None,
                    "region": None
                },
                "opening_hours": None,
                "price_range": None,
                "menu": []
            }

            # Lấy URL ảnh chính
            image_tag = soup.select_one(".main-image .img img")
            if image_tag and image_tag.get("src"):
                restaurant["Image"] = image_tag["src"].strip()

            # Tên nhà hàng
            name = soup.select_one("h1[itemprop='name']")
            restaurant["name"] = name.text.strip() if name else None

            # Các loại hình phụ (categories)
            restaurant["categories"] = [a.text.strip() for a in soup.select(".category-items a")]

            # Phong cách món ăn (cuisine)
            cuisine = soup.select_one(".cuisines-list a")
            restaurant["cuisine"] = cuisine.text.strip() if cuisine else None

            # Đối tượng phù hợp (audiences)
            audiences = soup.select_one(".audiences")
            if audiences:
                audiences_text = audiences.text.strip()
                audiences_list = audiences_text.replace("-", "").replace("\xa0", "").split(",")
                restaurant["audiences"] = [aud.strip() for aud in audiences_list]

            # Địa chỉ
            address_container = soup.select_one(".res-common-add")
            if address_container:
                street_address = address_container.select_one("span[itemprop='streetAddress']")
                restaurant["address"]["street"] = street_address.text.strip() if street_address else None

                address_locality = address_container.select_one("span[itemprop='addressLocality']")
                restaurant["address"]["locality"] = address_locality.text.strip() if address_locality else None

                address_region = address_container.select_one("span[itemprop='addressRegion']")
                restaurant["address"]["region"] = address_region.text.strip() if address_region else None

            # Giờ mở cửa
            opening_times = soup.select(".micro-timesopen span")
            if len(opening_times) >= 2:
                restaurant["opening_hours"] = opening_times[1].find_next_sibling("span").text.strip()

            # Giá
            price_range = soup.select_one(".res-common-minmaxprice span[itemprop='priceRange']")
            restaurant["price_range"] = price_range.text.strip() if price_range else None

            # Lấy đánh giá và bình luận
            ratings, comments = extract_ratings_and_comments(soup)
            restaurant["ratings"] = ratings
            restaurant["comments"] = comments
            # Menu
            menu_container = soup.select(".delivery-dishes-item")
            for item in menu_container:
                name_tag = item.select_one(".title-name-food .title-name")
                price_tag = item.select_one(".price")
                image_tag = item.select_one(".delivery-dishes-item-left img")

                if name_tag and price_tag and image_tag:
                    menu_item = {
                        "name": name_tag.text.strip(),
                        "price": price_tag.text.strip(),
                        "image_url": image_tag["src"].strip()
                    }
                    restaurant["menu"].append(menu_item)

            # Thêm vào danh sách
            List_restaurant_data.append(restaurant)
            data_extracted = True

        except Exception as e:
            print(f"Retry {retries + 1}/{MAX_RETRIES} - Lỗi: {e}")
            retries += 1
            time.sleep(random.randint(1, 2))  # Đợi trước khi thử lại

    if not data_extracted:
        print(f"Không thể thu thập dữ liệu từ {link} sau {MAX_RETRIES} lần thử.")

# Đóng trình duyệt
driver.quit()

In [13]:
write_dict_txt(List_restaurant_data, "Data2.txt")